# Descriptive statistics on simulated results
Below is a compact, Jupyter-friendly tutorial for creating descriptive statistics from apsimNGpy simulation outputs—plus an “alternate print” path using tabulate (remember to install it first).

# Prerequisites

# core need libraries
pip install apsimNGpy pandas

# optional pretty printing (used later)
pip install tabulate


Firs fetch the simulated output as a Pandas data frame


In [1]:
import pandas as pd
from apsimNGpy.core.apsim import ApsimModel

# Load a built-in template; omit the .apsimx suffix for templates
model = ApsimModel('Maize')

# If your file hasn’t been run yet, you may need:
model.run()

# Pull the “Report” table as a pandas DataFrame
df = model.get_simulated_output('Report')  # NOTE: correct quotes

# Optional: parse dates if present
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.head()


,CheckpointID,SimulationID,Zone,Clock.Today,Maize.Phenology.CurrentStageName,Maize.AboveGround.Wt,Maize.AboveGround.N,Yield,Maize.Grain.Wt,Maize.Grain.Size,Maize.Grain.NumberFunction,Maize.Grain.Total.Wt,Maize.Grain.N,Maize.Total.Wt,source_table
0,1,1,Field,1991-05-28 00:00:00,HarvestRipe,1603.309641,15.578250,8469.615813,846.961581,0.278267,3043.698222,846.961581,11.178291,1728.427114,Report
1,1,1,Field,1992-04-09 00:00:00,HarvestRipe,849.734144,9.684291,4674.514452,467.451445,0.273804,1707.246422,467.451445,6.226327,922.393712,Report
2,1,1,Field,1993-03-16 00:00:00,HarvestRipe,182.766781,1.861545,555.021350,55.502135,0.304067,182.532674,55.502135,0.752357,204.108770,Report
3,1,1,Field,1994-03-15 00:00:00,HarvestRipe,795.133784,8.344360,3504.274669,350.427467,0.226733,1545.553056,350.427467,4.886844,869.242545,Report
4,1,1,Field,1995-04-04 00:00:00,HarvestRipe,1525.129268,16.481579,7820.119109,782.011911,0.273512,2859.155304,782.011911,10.463854,1665.483701,Report


Whole-dataset descriptive statistics

In [2]:
# Select numeric columns only
num = df.select_dtypes('number')

# Basic summary
basic = num.agg(['count', 'mean', 'std', 'min', 'median', 'max']).T

# Add CV% and IQR
q = num.quantile([0.25, 0.75]).T.rename(columns={0.25: 'q25', 0.75: 'q75'})
summary = (
    basic
    .join(q)
    .assign(cv_percent=lambda x: (x['std'] / x['mean']) * 100)
)[['count','mean','std','cv_percent','min','q25','median','q75','max']]

summary.round(3)


,count,mean,std,cv_percent,min,q25,median,q75,max
CheckpointID,10.0,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000
SimulationID,10.0,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000
Maize.AboveGround.Wt,10.0,1225.100,561.129,45.803,182.767,840.745,1318.364,1587.793,1968.461
Maize.AboveGround.N,10.0,12.381,5.400,43.611,1.862,8.679,13.336,16.461,18.309
Yield,10.0,5636.530,2895.961,51.378,555.021,3578.665,6033.993,8239.976,8823.516
Maize.Grain.Wt,10.0,563.653,289.596,51.378,55.502,357.866,603.399,823.998,882.352
Maize.Grain.Size,10.0,0.285,0.024,8.339,0.227,0.275,0.295,0.299,0.305
Maize.Grain.NumberFunction,10.0,1986.771,992.188,49.940,182.533,1339.924,2123.083,2846.481,3043.698
Maize.Grain.Total.Wt,10.0,563.653,289.596,51.378,55.502,357.866,603.399,823.998,882.352
Maize.Grain.N,10.0,7.459,3.754,50.331,0.752,4.926,7.989,10.927,11.254


# Grouped descriptive statistics (e.g., by simulation or treatment)
For this example, we need the `ExperimentManager`` Class, where we can create some varying treatment levels



In [12]:
from apsimNGpy.core.experimentmanager import ExperimentManager
experiment = ExperimentManager(model ='Maize')
# init the experiment
experiment.init_experiment(permutation=True)

In [14]:
# add factors
# Population 
experiment.add_factor(specification='[Sow using a variable rule].Script.Population = 4, 6, 8, 10')
# Nitrogen fertilizers
experiment.add_factor(specification='[Fertilise at sowing].Script.Amount= 0, 100,150, 200, 250')

In [16]:
# run the model
experiment.run()


In [28]:
# Pick one or more grouping columns that exist in your Report table
df = experiment.get_simulated_output('Report')
df.sort_values(by =['Amount', 'Population'], inplace=True, ascending=True)
possible_groups = ['Population', 'Amount']
# describe stats by Population density
df.groupby('Population').describe()


CheckpointID                                    SimulationID        \
                  count mean  std  min  25%  50%  75%  max        count  mean   
Population                                                                      
10                 50.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         50.0   6.8   
4                  50.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         50.0   9.2   
6                  50.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         50.0  13.8   
8                  50.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         50.0  12.2   

            ... Maize.Grain.N            Maize.Total.Wt               \
            ...           75%        max          count         mean   
Population  ...                                                        
10          ...     12.453850  13.852686           50.0  1289.447656   
4           ...      8.475352   8.614249           50.0  1001.068741   
6           ...     10.464149  11.253935           50.0  1150.718224   
8           ...     11.484284  12.699701           50.0  1229.296565   

                                                                         \
                   std        min         25%          50%          75%   
Population                                                                
10          736.654239  54.190895  868.511797  1158.386221  1988.650295   
4           521.145254  98.643186  761.233392   956.896017  1426.426662   
6           640.234680  71.534517  853.918551  1083.421837  1690.019368   
8           691.742681  53.721510  887.970166  1116.849161  1853.933112   

                         
                    max  
Population               
10          2457.083319  
4           1708.187637  
6           2131.573319  
8           2317.939766  

[4 rows x 88 columns]

In [29]:
df.groupby("Amount").describe()

CheckpointID                                    SimulationID         \
              count mean  std  min  25%  50%  75%  max        count   mean   
Amount                                                                       
0              40.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         40.0   9.25   
100            40.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         40.0   9.50   
150            40.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         40.0  10.50   
200            40.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         40.0  10.25   
250            40.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0         40.0  13.00   

        ... Maize.Grain.N            Maize.Total.Wt                           \
        ...           75%        max          count         mean         std   
Amount  ...                                                                    
0       ...      2.656708   5.184583           40.0   456.329579  354.116147   
100     ...      9.415009  13.597501           40.0  1285.598734  541.853110   
150     ...     11.197194  13.852686           40.0  1376.463440  606.643390   
200     ...     11.197202  13.799295           40.0  1363.717576  614.948907   
250     ...     11.166839  13.799286           40.0  1356.054652  616.537160   

                                                                       
               min         25%          50%          75%          max  
Amount                                                                 
0        53.721510  171.719439   294.275738   744.446378  1204.283831  
100     218.291141  884.461655  1206.529259  1719.044626  2288.694976  
150     194.028708  946.894895  1381.721535  1863.525228  2457.083319  
200     193.824020  937.570871  1362.471767  1857.630645  2443.654397  
250     193.824020  936.024866  1352.608751  1848.713666  2443.652362  

[5 rows x 88 columns]

That is incredible but produces a lot of data, some of which is distractive, so, let target columns

In [30]:
# group by population and summarize the values by Yield
df.groupby("Population")['Yield'].describe()

,count,mean,std,min,25%,50%,75%,max
Population,,,,,,,,
10,50.0,5093.729388,3733.951073,0.000000,2252.624637,3627.256662,9198.856975,10881.111792
4,50.0,4212.521692,2169.197622,510.191575,3085.704298,3984.209202,6549.733680,6628.079544
6,50.0,4793.973779,2934.596152,241.859716,2895.653506,4105.005437,7821.202659,8823.530799
8,50.0,4963.756593,3337.240514,0.094526,2541.655824,3665.097489,8496.972606,9949.424774


In [31]:
# group by amount and summarize the values by Yield
df.groupby("Amount")['Yield'].describe()

,count,mean,std,min,25%,50%,75%,max
Amount,,,,,,,,
0,40.0,1738.677801,1236.562104,0.000000,862.028082,1250.717596,2576.077671,4490.541718
100,40.0,5179.234820,2679.061884,870.224406,3408.963611,4087.518866,7626.215922,10442.634519
150,40.0,5692.757501,3039.228130,481.337403,3315.165489,5992.029047,8477.449973,10881.111792
200,40.0,5628.385758,3078.335394,12.979605,3306.510890,5809.261385,8476.460558,10839.460880
250,40.0,5590.920934,3080.229281,13.881360,3290.721995,5574.223813,8402.826601,10839.453802


A big surprise here is that the minimum for the lower nitrogen(N) rate was actually higher than for the high N rates

# Time-based summaries (monthly/annual), optionally per group
For these examples, a date column is need to be available in the dataset

In [36]:
df['Date'] = pd.to_datetime(df['Clock.Today'])
time_groups = ['Population']

monthly = (
    df.set_index('Date')
      .groupby(time_groups)                    # ignored if empty
      .resample('YE')
      .mean(numeric_only=True)
      .reset_index()
)

monthly.head()


,Population,Date,CheckpointID,SimulationID,Maize.AboveGround.Wt,Maize.AboveGround.N,Yield,Maize.Grain.Wt,Maize.Grain.Size,Maize.Grain.NumberFunction,Maize.Grain.Total.Wt,Maize.Grain.N,Maize.Total.Wt
0,10,1991-12-31,1.0,6.8,1658.403538,15.372865,8179.939240,817.993924,0.220676,3690.828267,817.993924,10.670177,1781.102515
1,10,1992-12-31,1.0,6.8,775.023209,7.383173,3756.187249,375.618725,0.188569,1587.190677,375.618725,4.888130,841.899215
2,10,1993-12-31,1.0,6.8,242.859735,1.880137,471.620731,47.162073,0.148485,246.854345,47.162073,0.598606,269.890216
3,10,1994-12-31,1.0,6.8,726.619488,7.269006,2975.779427,297.577943,0.263948,1109.657831,297.577943,3.970384,784.147490
4,10,1995-12-31,1.0,6.8,1605.635285,16.433165,7891.747156,789.174716,0.220087,3498.771776,789.174716,10.510011,1748.366286


from tabulate import tabulate

# Pretty printing with tabulate (must be installed)


In [43]:
from tabulate import tabulate

#  Whole-dataset summary printed nicely
print(tabulate(summary.reset_index().rename(columns={'index':'variable'}).round(3),
               headers='keys', tablefmt='github', showindex=False))
data =df.copy()
data = data[['Population', 'Amount', 'Maize.Grain.Size', 'Yield']]
summary = data.groupby(['Population', 'Amount']).describe()
#  Grouped summary flattened and printed (if you created g_summary)

g_flat = summary.reset_index()  # bring group keys back as columns
print(tabulate(g_flat.round(3), headers='keys', tablefmt='github', showindex=False))


| variable   |   Maize.Grain.Size |    Yield |
|------------|--------------------|----------|
| count      |            200     |   200    |
| mean       |              0.246 |  4765.99 |
| std        |              0.074 |  3093.3  |
| min        |              0     |     0    |
| 25%        |              0.223 |  2535.99 |
| 50%        |              0.274 |  3894.86 |
| 75%        |              0.299 |  7819.71 |
| max        |              0.315 | 10881.1  |
|   ('Population', '') |   ('Amount', '') |   ('Maize.Grain.Size', 'count') |   ('Maize.Grain.Size', 'mean') |   ('Maize.Grain.Size', 'std') |   ('Maize.Grain.Size', 'min') |   ('Maize.Grain.Size', '25%') |   ('Maize.Grain.Size', '50%') |   ('Maize.Grain.Size', '75%') |   ('Maize.Grain.Size', 'max') |   ('Yield', 'count') |   ('Yield', 'mean') |   ('Yield', 'std') |   ('Yield', 'min') |   ('Yield', '25%') |   ('Yield', '50%') |   ('Yield', '75%') |   ('Yield', 'max') |
|----------------------|------------------|-------------

#  Export summaries to files

In [41]:
summary.to_csv("summary.csv")

Here’s a compact, Jupyter-friendly tutorial for creating **descriptive statistics** from **apsimNGpy** simulation outputs—plus an “alternate print” path using **`tabulate`** (remember to install it first).

---

# Descriptive statistics from APSIMNGpy outputs

## 0) Prerequisites

```bash
# core
pip install apsimNGpy pandas

# optional pretty printing (used later)
pip install tabulate
```

---

## 1) Get simulated output into a DataFrame

```python
import pandas as pd
from apsimNGpy.core.apsim import ApsimModel

# Load a built-in template; omit the .apsimx suffix for templates
model = ApsimModel('Maize')

# If your file hasn’t been run yet, you may need:
# model.run()

# Pull the “Report” table as a pandas DataFrame
df = model.get_simulated_output('Report')  # NOTE: correct quotes

# Optional: parse dates if present
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.head()
```

---

## 2) Whole-dataset descriptive statistics

```python
# Select numeric columns only
num = df.select_dtypes('number')

# Basic summary
basic = num.agg(['count', 'mean', 'std', 'min', 'median', 'max']).T

# Add CV% and IQR
q = num.quantile([0.25, 0.75]).T.rename(columns={0.25: 'q25', 0.75: 'q75'})
summary = (
    basic
    .join(q)
    .assign(cv_percent=lambda x: (x['std'] / x['mean']) * 100)
)[['count','mean','std','cv_percent','min','q25','median','q75','max']]

summary.round(3)
```

---

## 3) Grouped descriptive statistics (e.g., by simulation or treatment)

```python
# Pick one or more grouping columns that exist in your Report table
possible_groups = [c for c in ['SimulationName','Residue','Nitrogen','Tillage'] if c in df.columns]
group_cols = ['SimulationName'] if 'SimulationName' in df.columns else possible_groups  # choose what makes sense

if group_cols:
    grouped = df.groupby(group_cols, dropna=False)
    gnum = grouped[num.columns]

    # Named aggregations for common stats
    g_basic = gnum.agg(['count','mean','std','min','median','max'])

    # Compute q25, q75, and CV% per group
    g_q = gnum.quantile([0.25, 0.75]).unstack(level=-1)  # -> wide with (var, quantile)
    g_q.columns = [f"{var}_{qname}" for var, qname in g_q.columns]

    # Flatten MultiIndex on columns from g_basic
    g_basic.columns = [f"{var}_{stat}" for var, stat in g_basic.columns]

    # Join and append CV% columns
    g_summary = g_basic.join(g_q)

    for col in num.columns:
        g_summary[f"{col}_cv_percent"] = (g_summary[f"{col}_std"] / g_summary[f"{col}_mean"]) * 100

    # Optional: reorder columns per variable
    def order_for(v):
        return [f"{v}_{k}" for k in ["count","mean","std","cv_percent","min","q25","median","q75","max"] if f"{v}_{k}" in g_summary.columns]

    ordered_cols = sum((order_for(v) for v in num.columns), [])
    g_summary = g_summary[ordered_cols].round(3)

    g_summary
```

---

## 4) Time-based summaries (monthly/annual), optionally per group

```python
if 'Date' in df.columns:
    # Monthly mean of numeric variables for each SimulationName (if available)
    time_groups = []
    if 'SimulationName' in df.columns:
        time_groups = ['SimulationName']

    monthly = (
        df.set_index('Date')
          .groupby(time_groups)                    # ignored if empty
          .resample('M')
          .mean(numeric_only=True)
          .reset_index()
    )

    monthly.head()
```

---

## 5) “Another way” — pretty printing with `tabulate` (must be installed)

```python
from tabulate import tabulate

# 5a) Whole-dataset summary printed nicely
print(tabulate(summary.reset_index().rename(columns={'index':'variable'}).round(3),
               headers='keys', tablefmt='github', showindex=False))

# 5b) Grouped summary flattened and printed (if you created g_summary)
if group_cols:
    g_flat = g_summary.reset_index()  # bring group keys back as columns
    print(tabulate(g_flat.round(3), headers='keys', tablefmt='github', showindex=False))
```

> **Note:** `tabulate` is for pretty console/markdown tables. It doesn’t change the data—just the way it’s printed.

---

## Tips & troubleshooting

* If `df` is empty, ensure your APSIM file has a **Report** node and (if needed) call `model.run()` before `get_simulated_output('Report')`.
* If you see many `NaN` in summary tables, make sure you selected **numeric** columns (as above).
* For **multiple** Report tables, pass the exact one you want (e.g., `'Report'`, `'DailyReport'`) to `get_simulated_output(...)`.
* Prefer **explicit grouping** (e.g., `Population`, `Amount`) to avoid mixing different scenarios in the same statistics.


In [40]:
from apsimNGpy.core.config import apsim_version
from apsimNGpy import version
from apsimNGpy.settings import logger
print(f"Notebook generated by;\n APSIM version: `{apsim_version()}`\n apsimNGpy version {version.version}")

Notebook generated by;
 APSIM version: `APSIM 2025.8.7844.0`
 apsimNGpy version 0.39.9.15
